In [ ]:
from addict import Dict
from pathlib import Path
import numpy as np
import train as tr

data_dir = Path("/Users/kris/Desktop/teaching/mapping-vis/data/")
process_dir = data_dir / "processed"

args = Dict({
    "batch_size": 16,
    "epochs": 200,
    "lr": 0.001,
    "device": "cpu"
})

In [ ]:
from data import fetch_loaders
import matplotlib.pyplot as plt
%matplotlib inline

paths = {}
for split in ["train", "val"]:
    paths[split] = {}
    for v in ["x", "y"]:
        paths[split][v] = list(process_dir.glob(v + "*"))
        paths[split][v].sort()

loaders = fetch_loaders(paths, batch_size=args.batch_size)
x, y = next(iter(loaders["train"]))

In [ ]:
def plot_ims(x, y, N=10, channels = [2, 4, 5]):
    for i in range(N):
        xi = np.transpose(x[i, channels, :, :], (1, 2, 0))
        yi = np.transpose(y[i, [1, 1, 0], :, :], (1, 2, 0))
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 5))
        [axes[j].xaxis.set_visible(False) for j in [0, 1]]
        [axes[j].yaxis.set_visible(False) for j in [0, 1]]
        axes[0].imshow(0.5 * (1 - xi))
        axes[1].imshow(yi, alpha=0.5)
        plt.show()

In [ ]:
plot_ims(x, y)
plot_ims(x, y, channels=[11, 11, 11]) # elevation

In [ ]:
import torch.optim
import torch.cuda
from unet import Unet

model = Unet(13, 3, 4).to(args.device)
optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

for epoch in range(args.epochs):
    loss_d = {}
    for phase in ["train", "val"]:
        loss_d[phase] = tr.train_epoch(model, loaders[phase], optimizer, args.device, epoch)